In [25]:
using AeroAcoustics
using HDF5
using DelimitedFiles # for data export
using DSP # for Hanning window

In [26]:
time_file = "DTU_PLCT_NACA63018_trip_5PS_5SS_U0_50_AoA_0_TimeSeries.h5"
csm_file = "DTU_PLCT_NACA63018_trip_5PS_5SS_U0_50_AoA_0_octave-12_CsmEss.h5"

"DTU_PLCT_NACA63018_trip_5PS_5SS_U0_50_AoA_0_octave-12_CsmEss.h5"

## Task 1

In [27]:
# Collect time-series data and microphone geometry
function DTU_AIAAreadTimeseries(destfile::String)
    h5open(destfile, "r") do file
        MicrophoneData = read(file, "MicrophoneData")
        t = MicrophoneData["microphoneDataPa"]
        t = permutedims(t,reverse(1:ndims(t)))
        MetaData = read(file, "MetaData")
        micgeom = MetaData["ArrayAttributes"]["microphonePositionsM"]
        return t,micgeom
    end
    
end
t,micgeom = DTU_AIAAreadTimeseries(time_file);

In [28]:
# Collect meta data
ArrayAttributes = h5readattr(time_file,"MetaData/ArrayAttributes")
MeasurementData = h5readattr(time_file,"MeasurementData")
TestAttributes = h5readattr(time_file,"MetaData/TestAttributes")
MicrophoneData = h5readattr(time_file,"MicrophoneData/microphoneDataPa")

Dict{String, Any} with 2 entries:
  "sampleCount"  => 327680
  "sampleRateHz" => 16384

In [29]:
fs = MicrophoneData["sampleRateHz"]
n = 4096
CSM = csm(t;n=n,noverlap=div(n,2),fs=fs,win=DSP.hanning(n),scaling="spectrum");

In [30]:
# Locate indices of fc = 500Hz and 1000Hz:
fc = [500,1000]
f_idx = Int.(fc/(fs/n)).+1
for (fi,fc) in zip(f_idx,fc)
    CSM_out = CSM.arr[:,:,fi]
    writedlm("data/task1_DTU_AeroAcousticsjl_CSM_real_f_$(fc).csv",  real.(CSM_out), ';')
    writedlm("data/task1_DTU_AeroAcousticsjl_CSM_imag_f_$(fc).csv",  imag.(CSM_out), ';')
end


## Task 2

In [31]:
# To compute the psf, an environment must be defined, which requires a cross-spectral matrix.
# This can be instantiated with some fake data.
fc = [500,1000,2000,4000]
CSM = FreqArray(rand(84,84,4),fc)
E = Environment(
    CSM=CSM,
    micgeom=micgeom,
    z0 = TestAttributes["domainBoundsM"][1][3],
    dr = false,
    Nx = 41,
    Ny = 41,
    xlim = (-1,1),
    ylim = (-1,1),
    flim = (500,4000)
);
steeringvectors!(E)
p = psf(E);

In [32]:
for (fc,pif) in zip(fc,eachcol(p))
    writedlm("data/task2_DTU_AeroAcousticsjl_PSF_f_$(fc).csv",  reshape(pif,41,41), ';')
end

## Task 3

In [33]:
# Collect CSM-data and microphone geometry
function DTU_AIAAreadCSM(destfile::String)
    h5open(destfile, "r") do file
        CsmData = read(file, "CsmData")
        fc = CsmData["binCenterFrequenciesHz"]
        CSM = CsmData["csmReal"]+im*CsmData["csmImaginary"]
        CSM = permutedims(CSM,reverse(1:ndims(CSM)))
        MetaData = read(file, "MetaData")
        micgeom = MetaData["ArrayAttributes"]["microphonePositionsM"]
        return FreqArray(CSM,fc),micgeom
    end
    
end
CSM,micgeom = DTU_AIAAreadCSM(csm_file);

In [34]:
# Collect meta data
ArrayAttributes = h5readattr(csm_file,"MetaData/ArrayAttributes")
CsmData = h5readattr(csm_file,"CsmData")
MeasurementData = h5readattr(csm_file,"MeasurementData")
TestAttributes = h5readattr(csm_file,"MetaData/TestAttributes")

Dict{String, Any} with 5 entries:
  "domainBoundsM"       => @NamedTuple{1::Float64, 2::Float64, 3::Float64}[(var…
  "flowType"            => "kevlar"
  "machNumber"          => [0.150235, 0.0, 0.0]
  "coordinateReference" => "array center"
  "testDescription"     => "Airfoil in kevlar-walled wind tunnel"

In [35]:
# Setup environment
E = Environment(
    CSM=CSM,
    micgeom=micgeom,
    z0 = TestAttributes["domainBoundsM"][1][3],
    dr = true,
    Nx = 81,
    Ny = 41,
    xlim = (-2,2),
    ylim = (-1,1),
    flim = (400,4000)
);
steeringvectors!(E)

In [36]:
b = beamforming(E);

In [37]:
# Locate indices of fc = 500Hz, 1000Hz, 2000Hz, 4000Hz:
fc = [500,1000,2000,4000]
f_idx =  [argmin(abs.(E.fn .- fc)) for fc in fc] # finds index in E.fn that is closest to fc

for (fi,fc) in zip(f_idx,fc)
    writedlm("data/task3_DTU_AeroAcousticsjl_CBF_f_$(fc).csv",  reshape(SPL.(b[:,fi]),81,41), ';')
end

## Task 4

In [38]:
x = cleanSC(E;ϕ=0.5,maxiter=100);

In [39]:
# Locate indices of fc = 500Hz, 1000Hz, 2000Hz, 4000Hz:
fc = [500,1000,2000,4000]
f_idx =  [argmin(abs.(E.fn .- fc)) for fc in fc] # finds index in E.fn that is closest to fc

for (fi,fc) in zip(f_idx,fc)
    writedlm("data/task4_DTU_AeroAcousticsjl_CleanSC_f_$(fc).csv",  reshape(SPL.(x[:,fi]),81,41), ';')
end

## Task 5

In [18]:
# Setup environment with shear-layer
c = MeasurementData["speedOfSoundMPerS"]
Ma = TestAttributes["machNumber"][1]
E = Environment(
    CSM=CSM,
    micgeom=micgeom,
    z0 = TestAttributes["domainBoundsM"][1][3],
    dr = true,
    shear = true,
    Ma = -Ma,
    c = c,
    h = 1.5,
    Nx = 81,
    Ny = 41,
    xlim = (-2,2),
    ylim = (-1,1),
    flim = (400,4000)
);
steeringvectors!(E)

In [19]:
b = beamforming(E);

In [20]:
# Locate indices of fc = 500Hz, 1000Hz, 2000Hz, 4000Hz:
fc = [500,1000,2000,4000]
f_idx =  [argmin(abs.(E.fn .- fc)) for fc in fc] # finds index in E.fn that is closest to fc

for (fi,fc) in zip(f_idx,fc)
    writedlm("data/task5_DTU_AeroAcousticsjl_CBF_f_$(fc).csv",  reshape(SPL.(b[:,fi]),81,41), ';')
end

## Task 6

In [21]:
# Source integration region
limits = [−0.5,0.5,−0.4,0.4]
srcint_cbf = sourceintegration(b,E,limits);
writedlm("data/task6_DTU_AeroAcousticsjl_CBF_srcint.csv",  [E.fn SPL.(srcint_cbf)], ';')

## Task 7

In [22]:
x = cleanSC(E;ϕ=0.5,maxiter=100);

In [23]:
# Locate indices of fc = 500Hz, 1000Hz, 2000Hz, 4000Hz:
fc = [500,1000,2000,4000]
f_idx =  [argmin(abs.(E.fn .- fc)) for fc in fc] # finds index in E.fn that is closest to fc

for (fi,fc) in zip(f_idx,fc)
    writedlm("data/task7_DTU_AeroAcousticsjl_CleanSC_f_$(fc).csv",  reshape(SPL.(x[:,fi]),81,41), ';')
end

## Task 8

In [24]:
# Source integration region
limits = [−0.5,0.5,−0.4,0.4]
srcint_clean = sourceintegration(x,E,limits);
writedlm("data/task8_DTU_AeroAcousticsjl_CleanSC_srcint.csv",  [E.fn SPL.(srcint_clean)], ';')